# Elliptic Curve Cryptography

$E: y = x^{3} + Ax + B$ such that $4A^3 - 27B^2 \neq 0$

### Key generation for elliptic curves - Diffie-Hellman

The goal of key generation is to securely establish a channel and create a public key for symmetric key algorithms. It is used for encryption, password authentication for preventing man-in-the-middle attacks, and in forward secrecy based protocols to protect against key compromise by constantly generating new key pairs for each session. 

The function `elliptic_curve_dh_part1` produces a public key.

In [36]:
def elliptic_curve_dh_part1(p,A,B,P1,k):
    if (not(p.is_prime())):
        return 'Invalid'
    if ((4*power_mod(A, 3, p)+ 27*power_mod(B, 2, p)) % p == 0):
        return 'Invalid'
    if (not(power_mod(P[1],2,p) == (power_mod(P[0],3,p) + A*P[0] + B) % p)):
        return 'Invalid'
    F = GF(p)
    E = EllipticCurve(F,[A,B])
    dh_key = k*E(P)
    if (dh_key == E(0)):
        return 'Invalid'
    return (dh_key[0], dh_key[1])

### Elliptic Curve Diffie Hellman (ECDH)

The `ECDH algorithm` outputs a private/public key pair.

In [37]:
def elliptic_curve_dh_part2(p,A,B,b,P1):
    F = GF(p)
    E = EllipticCurve(F, [A,B])
    secret = b*E(P1) 
    return (ZZ(secret[0]),ZZ(secret[1]))

### Elliptic Curve Digital Signature Algorithm (ECDSA)

The function `elliptic_curve_dsa_part1` outputs a prime $p$ and a point on the ellipitic curve, $(x,y)$. 

In [38]:
def elliptic_curve_dsa_part1(p1,A,B):
    E = EllipticCurve(GF(p1),[A,B])
    p = E.cardinal().factor()[1][0]
    while(true): 
        G = E.random_element()
        if(G.additive_order() % p == 0):
            G = (G.additive_order() // p) * G
            x = ZZ(G[0]) 
            y = ZZ(G[1]) 
            return(p, (x,y))

This funciton `elliptic_curve_dsa_part2` outputs the signature, a tuple of pairs of integers, $((x,y),(xQ,yQ))$.

In [39]:
def elliptic_curve_dsa_part2(p1,p2,A,B,G,d,m):
    F = GF(p2)
    E = EllipticCurve(F,[A,B])
    G = E(G)
    if(G.additive_order() != p1):
        return 'Invalid'
    d = ZZ(randint(1,p1-1))
    Q = d*G
    if(m >= p1 or m >= p2):
        return 'Invalid'
    k = ephemeralkey(p1)
    R = k*G 
    x = ZZ(R[0]) % p 
    y = ZZ((power_mod(k,-1,p1) * (m + x*d)) % p1)
    xQ = ZZ(Q[0])
    yQ = ZZ(Q[1])
    return ((x,y),(xQ,yQ))

The function `elliptic_curve_dsa_part3` outputs a true or false boolean value depending on whether the signature is valid or not.

In [40]:
def elliptic_curve_dsa_part3(p1,A,B,G,p2,Q,m,validation):
    F = GF(p1)
    E = EllipticCurve(F,[A,B])
    G = E(G)
    Q = E(Q)
    F1 = GF(p2)
    x = F1(validation[0])
    y = F1(validation[1])
    m = F1(m)
    if(G.additive_order() != p2):
        return 'Invalid'
    C = ZZ((y^-1)*m)*G + ZZ((y^-1)*x)*Q
    if(ZZ(C[0]) % p2 == validation[0] % p):
        return True
    else:
        return False

### Lenstra's elliptic curve factorization

The function `elliptic_curve_lenstra_part1` outputs a value $B$ satisfying the equation, $E: y = x^{3} + Ax + B$, where $B < N$, or returns `Invalid` if there is no such $B$.

In [41]:
def elliptic_curve_lenstra_part1(N,P1,A):
    F = IntegerModRing(N)
    x = ZZ(P1[0])
    y = ZZ(P1[1])
    B = (y^2 - x^3 - A*x) % N
    E = EllipticCurve([F(A),F(B)])
    if(E == EllipticCurve([F(A),F(B)])):
        return ZZ(B % N)
    else:
        return 'Invalid'

The function `ellipitc_curve_lenstra_part2` outputs the list of points for which $kP$ goes through, where $k$ is an integer, and $P$ is a point on the elliptic curve. 

In [31]:
def ellipitc_curve_lenstra_part2(N,A,B,P1,k):
    F = IntegerModRing(N)
    E = EllipticCurve(F,[A,B])
    P1 = E(P1)
    Points = []
    P2 = P1
    for j in range (2,k+1):
        i = j
        P3 = E(0,1,0)
        P2 = Points[-1]
        while i != 0: 
            if i%2 == 1:
                try:
                    P3 = P3 + P2 
                except:
                    if P3[0] == P2[0]:
                        return ZZ(2*P3[1])
                    else:
                        return ZZ(P3[0] - P2[0])
            if (P2 == P2+P2):
                return P2
            else:
                return ZZ(2*P2[1])
            i = i//2
        Points.append(P3)
    return Points

The function `elliptic_curve_part3` combines the functions `elliptic_curve_lenstra_part1` and `elliptic_curve_lenstra_part2` by finding a $B < N$ and the list of points for which $kP$ goes through, where $k$ is an integer, and $P$ is a point on the elliptic curve. If all the computations of $kP$ go through, then `fails` is returned, else the gcd of $N$ and the integer which prevents the computations from going through is returned.

In [32]:
def elliptic_curve_part3(N,A,P1,k):
    B = elliptic_curve_lenstra_part1(N,P1,A)
    X = ellipitc_curve_lenstra_part2(N,A,B,P1,k)
    if type(X) == list:
        return 'fails'
    else:
        return gcd(X,N)

 ## Breaking the Merkle-Hellman knapsack

Bob’s public knapsack, given by $T = (t0, t1, . . . , tr−1)$, and Alice sends a message to Bob as a ciphertext block $C$ encrypted with $T$. If the attacker knows $T$ and $C$, they can break the cryptosystem if they can solve the matrix equation $TU = C$ where $U$ is a column matrix of $0s$ and $1s$.

Lattice reduction is used to solve many different types of combinatorial problems.

In 1982, Lenstra, Lenstra and Lovasz discovered the LLL algorithm for finding short vectors in a lattice.

The function `break_mh_part1` outputs a square matrix where the nonempty list $T$ of integers is appended to the matrix.

In [34]:
def break_mh_part1(T,C):
    n = len(T) - 1
    dimension = 2 + n
    m = identity_matrix(dimension-1)
    m2 = 2*m 
    rows1 = []
    for i in range(dim-1):
        rows1.append(1)
    rows1.append(C)
    v1 = vector((T))
    m3 = m2.augment(v1)
    m4 = m3.insert_row(dimension-1, (rows1))
    return m4

The function `break_mh_part2` finds a $b$ such that the product of $b$ and the matrix, $M$ (set using `break_mh_part1`), is the shortest row in the matrix, $ML$ (created by the `LLL` algorithm). The function outputs a binary sequence where $1$ corresponds to a $-1$ in $b$ and a $0$ corresponds to a $0$ in $b$.

In [35]:
def break_mh_part2(T,C):
    M = break_mh_part1(T,C)
    ML = M.LLL() 
    a = ML[0]
    aNorm = norm(a)
    for r in ML.rows():
         if norm(r) < aNorm:
            aNorm = norm(r)
            a = r
    b = M.solve_left(a)
    for i in range (len(v) - 1):
        lst = []
        if b[i] == -1:
            lst.append(ZZ(1))
        elif b[i] == 0:
            lst.append(ZZ(0))
    return lst